In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')#change dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.applications import imagenet_utils
from skimage import data, color, feature
import os
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2

Using TensorFlow backend.


In [0]:
df_train = pd.read_csv('/content/drive/My Drive/Lables1.csv')

In [0]:
df_train.head()


,ImageName,Level
0,1Img_874.jpg,1
1,1Img_875.jpg,1
2,1Img_876.jpg,1
3,1Img_877.jpg,1
4,1Img_878.jpg,1


In [0]:
targets_series = pd.Series(df_train['Level'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [0]:
one_hot_labels = np.asarray(one_hot)

In [0]:
im_size1 = 128
im_size2 = 128

In [0]:
x_train = []
y_train = []
x_test = []

In [0]:
#   i = 0 
#   #for folder in range(9):
#   for f, breed in tqdm(df_train.values):
#     if type(cv2.imread('IMAGES/{}'.format(f)))==type(None):
#       continue 
#     else:
#         img = cv2.imread('IMAGES/{}'.format(f))
#         label = one_hot_labels[i]
#         resizedImage = cv2.resize(img, (im_size1, im_size2))
#         img_grey_scale = color.rgb2gray(resizedImage)
#         canny = feature.canny(img_grey_scale, 3)
#         #resizedImagecanny = cv2.resize(canny, (im_size1, im_size2))
#         x_train.append(canny)
#         #np.concatenate(x_train, hog_vec[:])
#         y_train.append(label)
#         i += 1
# np.save('x_trainfocanny',x_train)
# np.save('y_trainfocanny',y_train)
# print('Done')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



Done


In [0]:
x_train = np.load('x_trainfocanny.npy')
y_train = np.load('y_trainfocanny.npy')
x_train = np.expand_dims(x_train, axis=-1) 

In [0]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.

In [0]:
print(x_train_raw.shape)
print(y_train_raw.shape)

(13765, 128, 128, 1)
(13765, 8)


In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.1, random_state=1)

In [0]:
num_class = y_train_raw.shape[1]

In [0]:
from keras.applications.resnet50 import ResNet50

In [0]:
base_model = ResNet50(weights = None, include_top=False, input_shape=(im_size1, im_size2, 1))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.8)(x)
#x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
#predictions = Dense(num_class, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

#model.compile(loss='categorical_crossentropy', 
              # optimizer='rmsprop', 
              # metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adam', 
#               metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', 
              optimizer='SGD', 
              metrics=['accuracy'])


callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), verbose=1)

Train on 12388 samples, validate on 1377 samples
Epoch 1/20
12388/12388 [==============================] - 155s 12ms/step - loss: 1.2426 - accuracy: 0.5554 - val_loss: 2.3060 - val_accuracy: 0.3195
Epoch 2/20
12388/12388 [==============================] - 136s 11ms/step - loss: 0.7095 - accuracy: 0.7641 - val_loss: 4.1179 - val_accuracy: 0.3195
Epoch 3/20
12388/12388 [==============================] - 136s 11ms/step - loss: 0.5003 - accuracy: 0.8359 - val_loss: 3.4226 - val_accuracy: 0.3428
Epoch 4/20
12388/12388 [==============================] - 136s 11ms/step - loss: 0.3992 - accuracy: 0.8735 - val_loss: 0.3897 - val_accuracy: 0.8627
Epoch 5/20
12388/12388 [==============================] - 136s 11ms/step - loss: 0.2845 - accuracy: 0.9086 - val_loss: 0.4389 - val_accuracy: 0.8584
Epoch 6/20
12388/12388 [==============================] - 136s 11ms/step - loss: 0.2161 - accuracy: 0.9286 - val_loss: 0.2880 - val_accuracy: 0.9099
Epoch 7/20
12388/12388 [==============================] -